In [3]:
from pathlib import Path
import os
import re

In [2]:
in_dir = Path('../data/out/dbpedia-live')
out_dir = Path('../data/out/dbpedia-fixed')

In [33]:
rx_gregorian = re.compile(' "-(\d{1,3})"\^\^<http:\/\/www\.w3\.org\/2001\/XMLSchema#gYear>')
rx_bad_uri = re.compile(' <(https?):\/?([^\/])')
rx_bad_uri_2 = re.compile(' <(https?):\/{3,}([^\/])')
rx_bad_uri_3 = re.compile(' <http[^>]{,10}https?:[^>]*>')
rx_bad_uri_4 = re.compile(' <http[^>]+https?:\/?[^/][^>]+>')
rx_bad_uri_5 = re.compile(' <https?://(-[^>]+|[^/>]+[-:])>')
rx_bad_uri_6 = re.compile(' <https?:/{1,}>')
what_even_is_this = re.compile(' <[^>]+@[^>]*>')
# This is not perfect, but the ways of broken Unicode are many and varied.
# Too complex for me.
# The few random characters outside the main "bad" range are deprecated Unicode chars, apparently.
# Unfortunately, there is more of them. Thankfully not in this dataset.
rx_bad_utf_iri = re.compile(' <[^>]+[\u0673\u0f77\u0f79\ud7ff-\U0010FFFD][^>]*>')
rx_bad_infinity = re.compile('"(-?)Inf[^"]*"(\^\^<http://www\.w3\.org/2001/XMLSchema#(double|float)>)')

for d1 in sorted(os.listdir(in_dir)):
    for d2 in sorted(os.listdir(in_dir / d1)):
        (out_dir / d1 / d2).mkdir(parents=True, exist_ok=True)
        for f in sorted(os.listdir(in_dir / d1 / d2)):
            if not f.endswith('.ttl'):
                continue
            out_path = out_dir / d1 / d2 / f
            with open(in_dir / d1 / d2 / f) as in_f:
                with open(out_path, 'wt') as out_f:
                    for line in in_f:
                        if what_even_is_this.search(line) or rx_bad_utf_iri.search(line) or \
                                rx_bad_uri_3.search(line) or rx_bad_uri_4.search(line) or \
                                rx_bad_uri_5.search(line) or rx_bad_uri_6.search(line):
                            continue
                        # Fix gregorian negative years... they must have 4 digits FOR WHATEVER REASON
                        # standards, man.
                        line = rx_gregorian.sub(
                            lambda m: f' "-{int(m.group(1)):04}"^^<http://www.w3.org/2001/XMLSchema#gYear>',
                            line
                        )
                        line = rx_bad_uri.sub(r' <\1://\2', line)
                        line = rx_bad_uri_2.sub(r' <\1://\2', line)
                        line = rx_bad_infinity.sub(r'"\1INF"\2', line)
                        # unicode replacement character -- ??!?!?
                        line = line.replace('\uFFFD', '?')
                        out_f.write(line)

            if int(f[:-4]) % 10_000 == 0:
                print(f[:-4])

0000000000
0000010000
0000020000
0000030000
0000040000
0000050000
0000060000
0000070000
0000080000
0000090000
0000100000
0000110000
0000120000
0000130000
0000140000
0000150000
0000160000


In [17]:
# actual thing that came out of wikipedia
t = 'What_tнe⃗_♯$*!_D⃗𝞱_𝓌Σ_(k)πow!%3F&oldid=480710914>'
for c in t:
    # c and ord(c) in hex
    print(c, hex(ord(c)))

W 0x57
h 0x68
a 0x61
t 0x74
_ 0x5f
t 0x74
н 0x43d
e 0x65
⃗ 0x20d7
_ 0x5f
♯ 0x266f
$ 0x24
* 0x2a
! 0x21
_ 0x5f
D 0x44
⃗ 0x20d7
𝞱 0x1d7b1
_ 0x5f
𝓌 0x1d4cc
Σ 0x3a3
_ 0x5f
( 0x28
k 0x6b
) 0x29
π 0x3c0
o 0x6f
w 0x77
! 0x21
% 0x25
3 0x33
F 0x46
& 0x26
o 0x6f
l 0x6c
d 0x64
i 0x69
d 0x64
= 0x3d
4 0x34
8 0x38
0 0x30
7 0x37
1 0x31
0 0x30
9 0x39
1 0x31
4 0x34
> 0x3e


In [25]:
# this one makes my cursor go crazy
t = 'title=ٳ&action=history>'
for c in t:
    print(c, hex(ord(c)))


t 0x74
i 0x69
t 0x74
l 0x6c
e 0x65
= 0x3d
ٳ 0x673
& 0x26
a 0x61
c 0x63
t 0x74
i 0x69
o 0x6f
n 0x6e
= 0x3d
h 0x68
i 0x69
s 0x73
t 0x74
o 0x6f
r 0x72
y 0x79
> 0x3e


In [32]:
t = 'le=ཹ&ol'
for c in t:
    print(c, hex(ord(c)))


l 0x6c
e 0x65
= 0x3d
ཹ 0xf79
& 0x26
o 0x6f
l 0x6c
